# Breast cancer molecular subtyping using hypergraphs

In [ ]:
#the feature reduction that slects the best features using ramdom forest classifier

In [4]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from numpy import savetxt
import numpy as np

data = pd.read_csv('cnv_old.csv')
y = pd.read_csv('labels_old.csv')                  
# x = data.drop('SAMPLE_ID',axis = 1,inplace = True) 
# x = data.drop('SUBTYPE',axis = 1,inplace = True)
x=data
x = x.astype(np.float64)
np.ravel(y,order='c')
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=42)
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100),max_features=1000)
sel.fit(x_train, y_train)
sel.get_support()
selected_feat= x_train.columns[(sel.get_support())]
print(selected_feat)
methx_1=x[selected_feat]
savetxt('cnv_old_1000.csv', methx_1, delimiter=',')

C:\Users\user1\Anaconda3\lib\site-packages\sklearn\feature_selection\_from_model.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)


Index(['ABCB8', 'ABHD13', 'ABL2', 'ACSL4', 'ACSM2A', 'ACTG1', 'ACVRL1', 'ADA',
       'ADCY4', 'ADCY7',
       ...
       'ZNF592', 'ZNF608', 'ZNF645', 'ZNF652', 'ZNF740', 'ZNF75A', 'ZNF770',
       'ZNF830', 'ZNRF2', 'ZSCAN29'],
      dtype='object', length=1000)


In [ ]:
#the deep embedded clustering that groups the cancer dataset into subtypes

In [1]:
import keras.backend as K
from keras.engine.topology import Layer, InputSpec
from keras.layers import Dense, Input
from keras.models import Model
from keras.optimizers import SGD
from keras.optimizers import Adam
from sklearn.metrics import silhouette_score
from keras.initializers import VarianceScaling
from sklearn.cluster import KMeans
from numpy import savetxt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from sklearn import preprocessing
class ClusteringLayer(Layer):
    '''
    Clustering layer converts input sample (feature) to soft label, i.e. a vector that represents the probability of the
    sample belonging to each cluster. The probability is calculated with student's t-distribution.
    '''

    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights
        self.input_spec = InputSpec(ndim=2)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight(name='clusters', shape=(self.n_clusters, input_dim), initializer='glorot_uniform') 
        
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        ''' 
        student t-distribution, as used in t-SNE algorithm.
        It measures the similarity between embedded point z_i and centroid µ_j.
                 q_ij = 1/(1+dist(x_i, µ_j)^2), then normalize it.
                 q_ij can be interpreted as the probability of assigning sample i to cluster j.
                 (i.e., a soft assignment)
       
        inputs: the variable containing data, shape=(n_samples, n_features)
        
        Return: student's t-distribution, or soft labels for each sample. shape=(n_samples, n_clusters)
        '''
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1)) # Make sure all of the values of each sample sum up to 1.
        
        return q

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters}
        base_config = super(ClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

def autoencoder(dims, act='relu', init='glorot_uniform'):
    """
    Fully connected symmetric auto-encoder model.
  
    dims: list of the sizes of layers of encoder like [500, 500, 2000, 10]. 
          dims[0] is input dim, dims[-1] is size of the latent hidden layer.

    act: activation function
    
    return:
        (autoencoder_model, encoder_model): Model of autoencoder and model of encoder
    """
    n_stacks = len(dims) - 1
    
    input_data = Input(shape=(dims[0],), name='input')
    x = input_data
    
    # internal layers of encoder
    for i in range(n_stacks-1):
        x = Dense(dims[i + 1], activation=act, kernel_initializer=init, name='encoder_%d' % i)(x)

    # latent hidden layer
    encoded = Dense(dims[-1], kernel_initializer=init, name='encoder_%d' % (n_stacks - 1))(x)

    x = encoded
    # internal layers of decoder
    for i in range(n_stacks-1, 0, -1):
        x = Dense(dims[i], activation=act, kernel_initializer=init, name='decoder_%d' % i)(x)

    # decoder output
    x = Dense(dims[0], kernel_initializer=init, name='decoder_0')(x)
    
    decoded = x
    
    autoencoder_model = Model(inputs=input_data, outputs=decoded, name='autoencoder')
    encoder_model     = Model(inputs=input_data, outputs=encoded, name='encoder')
    
    return autoencoder_model, encoder_model


def target_distribution(q):
    weight = q ** 2 / q.sum(0)
    return (weight.T / weight.sum(1)).T

np.random.seed(10)
data = pd.read_csv('newcnv_1000.csv')
data.fillna(value=0, inplace = True)
data = data.astype(np.float64)
scaler = MinMaxScaler() 
numeric_columns = data.columns.values.tolist()
data[numeric_columns] = scaler.fit_transform(data[numeric_columns])

#autoencoder

n_clusters = 5# 3 - 0.291, 4 - 0.280, 5 - 0.228, 6 - 0.227, 10 - 0.192
n_epochs   = 100
batch_size = 128
x = data.values
x = preprocessing.normalize(x)
# x=preprocessing.scale(x)
kmeans = KMeans(n_clusters=n_clusters)
y_pred_kmeans = kmeans.fit_predict(x)
dims = [x.shape[-1], 500, 500, 1000, 300] 
init = VarianceScaling(scale=1. / 3., mode='fan_in',distribution='uniform')
pretrain_optimizer = SGD(lr=1, momentum=0.9)
pretrain_epochs = n_epochs
batch_size = batch_size
save_dir = './results'
autoencoder, encoder = autoencoder(dims, init=init)
autoencoder.compile(optimizer=pretrain_optimizer, loss='mse')
autoencoder.fit(x, x, batch_size=batch_size, epochs=pretrain_epochs)
autoencoder.save_weights(save_dir + '/ae_weights.h5')
autoencoder.load_weights(save_dir + '/ae_weights.h5')
    
clustering_layer = ClusteringLayer(n_clusters, name='clustering')(encoder.output)
model = Model(inputs=encoder.input, outputs=clustering_layer)
model.compile(optimizer=SGD(0.05, 0.9), loss='kld')

kmeans = KMeans(n_clusters=n_clusters, n_init=20)
y_pred = kmeans.fit_predict(encoder.predict(x)).astype(np.float64)
y_pred_last = np.copy(y_pred)

model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])

# computing an auxiliary target distribution

loss = 0
index = 0
maxiter = 1000 # 8000
update_interval = 100 # 140
index_array = np.arange(x.shape[0])
tol = 0.001 # 0.001tolerance threshold to stop training
for ite in range(int(maxiter)):
    if ite % update_interval == 0:
        q = model.predict(x, verbose=0)
        p = target_distribution(q)  # update the auxiliary target distribution p

    idx = index_array[index * batch_size: min((index+1) * batch_size, x.shape[0])]
    loss = model.train_on_batch(x=x[idx], y=p[idx])
    index = index + 1 if (index + 1) * batch_size <= x.shape[0] else 0

model.save_weights(save_dir + '/DEC_model_final.h5')
model.load_weights(save_dir + '/DEC_model_final.h5')

# Eval.
q = model.predict(x, verbose=0)
p = target_distribution(q)  # update the auxiliary target distribution p

# evaluate the clustering performance
y_pred = q.argmax(1)
x_embedded = TSNE(n_components=2).fit_transform(x)
x_embedded.shape

# vis_x = x_embedded[:, 0]
# vis_y = x_embedded[:, 1]

K.set_floatx('float64')
init_1 = VarianceScaling(scale=1. / 3., mode='fan_in',distribution='uniform')
autoencoder.load_weights(save_dir + '/ae_weights.h5')
clustering_layer = ClusteringLayer(n_clusters, name='clustering')(encoder.output)
model = Model(inputs=encoder.input, outputs=[clustering_layer, autoencoder.output])


kmeans = KMeans(n_clusters=n_clusters, n_init=20)
y_pred = kmeans.fit_predict(encoder.predict(x))

model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])
y_pred_last = np.copy(y_pred)

model.compile(loss=['kld', 'mse'], loss_weights=[0.1, 1], optimizer=pretrain_optimizer)
for ite in range(int(maxiter)):
    if ite % update_interval == 0:
        q, _  = model.predict(x, verbose=0)
        p = target_distribution(q)  # update the auxiliary target distribution p

        # evaluate the clustering performance
        y_pred = q.argmax(1)

        # check stop criterion chnage to float32 for cnv
        delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
        y_pred_last = np.copy(y_pred)
        if ite > 0 and delta_label < tol:
            print('delta_label ', delta_label, '< tol ', tol)
            print('Reached tolerance threshold. Stopping training.')
            break
    idx = index_array[index * batch_size: min((index+1) * batch_size, x.shape[0])]
    loss = model.train_on_batch(x=x[idx], y=[p[idx], x[idx]])
    index = index + 1 if (index + 1) * batch_size <= x.shape[0] else 0

model.save_weights(save_dir + '/b_DEC_model_final.h5')
model.load_weights(save_dir + '/b_DEC_model_final.h5')
# evaluation of model prediction 
data = pd.read_csv('newcnv_1000.csv')
data.fillna(value=0, inplace = True)
data = data.astype(np.float64)
x=data.values
q, _ = model.predict(x, verbose=0)
p = target_distribution(q)  # update the auxiliary target distribution p

# evaluate the clustering performance
y_pred = q.argmax(1)
savetxt('cnv_train_test_diff.csv', y_pred, delimiter=',')


ERROR! Session/line number was not unique in database. History logging moved to new session 23
Epoch 1/100
980/980 [==============================] - 0s 428us/step - loss: 9.6722e-04
Epoch 2/100
980/980 [==============================] - 0s 366us/step - loss: 8.1265e-04
Epoch 3/100
980/980 [==============================] - 0s 364us/step - loss: 6.2189e-04
Epoch 4/100
980/980 [==============================] - 0s 384us/step - loss: 4.5907e-04
Epoch 5/100
980/980 [==============================] - 0s 372us/step - loss: 3.3854e-04
Epoch 6/100
980/980 [==============================] - 0s 402us/step - loss: 2.5531e-04
Epoch 7/100
980/980 [==============================] - 0s 389us/step - loss: 1.9988e-04
Epoch 8/100
980/980 [==============================] - 0s 374us/step - loss: 1.6369e-04
Epoch 9/100
980/980 [==============================] - 0s 381us/step - loss: 1.4027e-04
Epoch 10/100
980/980 [==============================] - 0s 358us/step - loss: 1.2531e-04
Epoch 11/100
980/980 [==

C:\Users\Dassen Sathan\anaconda3\envs\keras_env\lib\site-packages\sklearn\manifold\_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
C:\Users\Dassen Sathan\anaconda3\envs\keras_env\lib\site-packages\sklearn\manifold\_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


delta_label  0.0 < tol  0.001
Reached tolerance threshold. Stopping training.


In [ ]:
#finding the optimal number of clusters using hypergraph partitioning

In [3]:
import numpy as np
import pandas as pd
from scipy.special import comb as choose
import itertools
from scipy.stats import entropy
import math
import collections
from scipy.spatial import distance
import networkx as nx
import time

# return: number of nodes (recall: 0-based), and
# number of edges of each cardinality
def H_size(H):
    M = len(H)
    m = []
    n = 0
    for i in range(M):
        m.append(len(H[i]))
        if(len(H[i]) > 0):
            j = max(set.union(*H[i]))

            if j > n:
                n = j
    return n+1, m

# vertex d-degrees for each d

def d_Degrees(H, n, m):
    M = len(H)
    d = [[]]*M
    for i in range(M):
        if(m[i] > 0):
            #x = [y for x in H[i] for y in list(x)]
            x = [y for x in H[i] for y in list(x)]
            y = [x.count(i) for i in range(n)]
            d[i] = y
    return d

# vertex total degrees
def Degrees(H, n, m, d):
    M = len(H)
    D = [0]*n
    for i in range(M):
        if(m[i] > 0):
            for j in range(n):
                D[j] = D[j] + d[i][j]
    return D

##########################################################
##entropy calculation
def entropy1(labels, base=None):
  value,counts = np.unique(labels, return_counts=True)
  return entropy(counts, base=base)

def conductance_partition(H,A):
    print(nx.conductance(H,A))

def dist_calc(a,b):
    euclidean=0
    for i in a:
        for j in b:
            euclidean+=(i-j)**2
    return euclidean**0.5

######

# edge contribution: given (H,A,m)
def EdgeContribution(H, A, m):
    ec = 0
    for i in range(len(H)):
        for j in range(len(H[i])):
            for k in range(len(A)):
                if(H[i][j].issubset(A[k])):
                    ec = ec + 1
                    break
    ec = ec / sum(m)
    return ec

##########################################################

def d_DegreeTax(A, m, d):
    dt = 0
    for i in range(len(m)):
        if (m[i] > 0):
            S = 0
            for j in range(len(A)):
                s = 0
                for k in A[j]:
                    s = s + d[i][k]
                s = s ** i
                S = S + s
            S = S / (i**i * m[i]**(i-1) * sum(m))
            dt = dt + S
    return dt

# degree tax - with degrees as null model
def DegreeTax(A, m, D):
    vol = sum(D)
    M = sum(m)
    # vol(A_i)'s
    volA = [0]*len(A)
    for i in range(len(A)):
        for j in A[i]:
            volA[i] = volA[i] + D[j]
        volA[i] = volA[i] / vol
    # sum over d
    S = 0
    for i in range(len(m)):
        if (m[i] > 0):
            x = sum([a**i for a in volA]) * m[i] / M
            S = S + x
    return S


############################
def newPart(A, s):
    P = []
    for i in range(len(A)):
        if(len(s.intersection(A[i])) == 0):
            P.append(A[i])
        else:
            s = s.union(A[i])
    P.append(s)
    return P

##########################################################


def cnmAlgo(H,verbose=False, ddeg=False):
    # get degrees from H
    n, m = H_size(H)
    d = d_Degrees(H, n, m)
    D = Degrees(H, n, m, d)
    # get all edges in a list
    e = []
    for i in range(len(H)):
        e.extend(H[i])
    

    # initialize modularity, partition
    A_opt = []
    for i in range(n):
        A_opt.extend([{i}])
        entropy_value=entropy1(A_opt)
           
        if (entropy_value !=0):
            entropy_log=math.log(entropy_value,2)
        else:
            entropy_log=0.0001
        
        q_opt = (EdgeContribution(H, A_opt, m) - DegreeTax(A_opt, m, D))+(entropy_log/len(A_opt)) 
        
       
    # e contains the edges NOT yet in a part
    while len(e) > 0:
        q0 = -1
        e0 = -1
        if verbose:
            print('best overall:', q_opt, 'edges left: ', len(e))
        # pick best edge to add .. this is slow as is!
        for i in range(len(e)):
            P = newPart(A_opt, e[i])
            
            P_list= list(P)
            entropy_value=entropy1(P_list)
           
            if (entropy_value !=0):
                entropy_log=math.log(entropy_value,2)
            else:
                entropy_log=0.0001
            #q = (EdgeContribution(H, P, m) - DegreeTax(P, m, D))*(entropy_log/len(P))
            q = (EdgeContribution(H, P, m) - DegreeTax(P, m, D))+(entropy_log/len(P))
            
            if q > q0:
                e0 = i
                q0 = q
        # add best edge found if any
        if(q0 > q_opt):
            q_opt = q0
            A_opt = newPart(A_opt, e[e0])
            # remove all 'active' edges
            r = []
            for i in range(len(e)):
                for j in range(len(A_opt)):
                    if(e[i].issubset(A_opt[j])):
                        r.append(e[i])
                        break
            for i in range(len(r)):
                e.remove(r[i])
        # early stop if no immediate improvement
        else:
            break
    return q_opt, A_opt



# Map vertices 0 .. n-1 to their respective 0-based part number


def PartitionLabels(P):
    n = 0
    for i in range(len(P)):
        n = n + len(P[i])
    label = [-1]*n
    for i in range(len(P)):
        l = list(P[i])
        for j in range(len(l)):
            label[l[j]] = i
    return label

##########################################################

# format Hypergraph given list of hyperedges (list of sets of 0-based integers)


    
def list2H(h):
    ml = max([len(x) for x in h])
    
    H = [[]]*(ml+1)
    for i in range(ml+1):
        H[i] = []
   
    for i in range(len(h)):
        l = len(h[i])
        H[l].append(h[i])
    return H

# two section modularity


def hcutPlus(H,P):
    hc = [-1]*len(H)
    S = 0
    L = 0
    for i in range(len(H)):
        l = len(H[i])
        L = L + l
        s = 0
        for j in range(len(P)):            
            s = s + sum([x < set(P[j]) for x in H[i] ])
        S = S + s
        if l>0:
            hc[i] = (l-s)/l
    return hc


df =  pd.read_csv('edges_array_newrna.csv',header=None)
h = df.apply(set, axis=1)

h_int=list()
for i in range(len(h)):
    h_int= [np.int(x) for x in h[i] if x==x]
    h_int=set(h_int)
    h[i]=h_int

h_2h=list2H(h)
qC, PC = cnmAlgo(h_2h,verbose=True)
print(len(PC))
np.savetxt("n5_newrna.csv",PartitionLabels(PC), delimiter=",")


C:\Users\user1\AppData\Local\Temp/ipykernel_1584/1393304641.py:254: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  h_int= [np.int(x) for x in h[i] if x==x]


best overall: 0.006576622021093001 edges left:  15
best overall: 0.20908322987511513 edges left:  12
best overall: 0.41403131151079786 edges left:  9
best overall: 0.6266449781661371 edges left:  6
best overall: 0.8461660384380215 edges left:  3
5


In [ ]:
#Evaluating the accuracy of the results by comparing it with the gold standard for the dataset

In [2]:
import argparse
import scipy.special
import csv
import numpy as np
import pandas as pd

# Get paths to clustering result and gold standard
clustered_file = 'newrna_200_n5_2.csv'
gold_standard_file = 'ground_test_label2.csv'

print("\nStarting evaluate.py...")
print("Clustering results file:", clustered_file)
print("gold standard file:", gold_standard_file)

# Get the number of clusters from the gold standard
#---------------------------------------------------------
gold_standard_n_clusters = 0

all_gold_standard_clusters_list = []

with open(gold_standard_file) as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        all_gold_standard_clusters_list.append(row[1])
        
gold_standard_clusters_list = list(set(all_gold_standard_clusters_list))
gold_standard_n_clusters = len(gold_standard_clusters_list)

print("\nNumber of clusters available in the gold standard: ", gold_standard_n_clusters)


# Get the gold standard
#----------------------------
gold_standard_clusters = [[] for x in range(gold_standard_n_clusters)]

gold_standard_count = 0

with open(gold_standard_file) as contig_clusters:
    readCSV = csv.reader(contig_clusters, delimiter=',')
    for row in readCSV:
        gold_standard_count += 1
        contig = row[0]
        bin_num = gold_standard_clusters_list.index(row[1])
        gold_standard_clusters[bin_num].append(contig)

print("Number of objects available in the gold standard: ", gold_standard_count)

# Get the number of clusters from the initial clustering result
#---------------------------------------------------------
n_clusters = 0

all_clusters_list = []

with open(clustered_file) as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        all_clusters_list.append(row[1])
        
clusters_list = list(set(all_clusters_list))
n_clusters = len(clusters_list)

print("Number of clusters available in the clustering result: ", n_clusters)


# Get initial clustering result
#----------------------------
clusters = [[] for x in range(n_clusters)]

clustered_count = 0
clustered_objects = []

with open(clustered_file) as contig_clusters:
    readCSV = csv.reader(contig_clusters, delimiter=',')
    for row in readCSV:
        clustered_count += 1
        contig = row[0]
        bin_num = clusters_list.index(row[1])
        clusters[bin_num].append(contig)
        clustered_objects.append(contig)

print("Number of objects available in the clustering result: ", len(clustered_objects))

# Functions to determine precision, recall, F1-score and ARI
#------------------------------------------------------------

# Get precicion
def getPrecision(mat, k, s, total):
    sum_k = 0
    for i in range(k):
        max_s = 0
        for j in range(s):
            if mat[i][j] > max_s:
                max_s = mat[i][j]
        sum_k += max_s
    return sum_k/total*100

# Get recall
def getRecall(mat, k, s, total, unclassified):
    sum_s = 0
    for i in range(s):
        max_k = 0
        for j in range(k):
            if mat[j][i] > max_k:
                max_k = mat[j][i]
        sum_s += max_k
    return sum_s/(total+unclassified)*100

# Get ARI
def getARI(mat, k, s, N):
    t1 = 0    
    for i in range(k):
        sum_k = 0
        for j in range(s):
            sum_k += mat[i][j]
        t1 += scipy.special.binom(sum_k, 2)
        
    t2 = 0
    for i in range(s):
        sum_s = 0
        for j in range(k):
            sum_s += mat[j][i]
        t2 += scipy.special.binom(sum_s, 2)
        
    t3 = t1*t2/scipy.special.binom(N, 2)
    
    t = 0
    for i in range(k):
        for j in range(s):
            t += scipy.special.binom(mat[i][j], 2)
        
    ari = (t-t3)/((t1+t2)/2-t3)*100
    return ari

# Get F1-score
def getF1(prec, recall):
    return 2*prec*recall/(prec+recall)


# Determine precision, recall, F1-score and ARI for clustering result
#------------------------------------------------------------------

total_clustered = 0

clusters_species = [[0 for x in range(gold_standard_n_clusters)] for y in range(n_clusters)]

for i in range(n_clusters):
    for j in range(gold_standard_n_clusters):
        n = 0
        for k in range(clustered_count):
            if clustered_objects[k] in clusters[i] and clustered_objects[k] in gold_standard_clusters[j]:
                n+=1
                total_clustered += 1
        clusters_species[i][j] = n

print("Number of objects available in the clustering result that are present in the gold standard:", total_clustered)

my_precision = getPrecision(clusters_species, n_clusters, gold_standard_n_clusters, total_clustered)
my_recall = getRecall(clusters_species, n_clusters, gold_standard_n_clusters, total_clustered, (gold_standard_count-total_clustered))
my_ari = getARI(clusters_species, n_clusters, gold_standard_n_clusters, total_clustered)
my_f1 = getF1(my_precision, my_recall)

print("\nEvaluation Results:")
print("Precision =", my_precision)
print("Recall =", my_recall)
print("F1-score =", my_f1)
print("ARI =", my_ari)

print()


Starting evaluate.py...
Clustering results file: newrna_200_n5_2.csv
gold standard file: ground_test_label2.csv

Number of clusters available in the gold standard:  5
Number of objects available in the gold standard:  392
Number of clusters available in the clustering result:  5
Number of objects available in the clustering result:  392
Number of objects available in the clustering result that are present in the gold standard: 392

Evaluation Results:
Precision = 73.21428571428571
Recall = 50.0
F1-score = 59.42028985507246
ARI = 30.99374315281039

ERROR! Session/line number was not unique in database. History logging moved to new session 26
